In [2]:
import os
import time


In [20]:
try:
    %load_ext autotime
except:
    !pip install -q ipython-autotime
    %load_ext autotime

time: 888 µs (started: 2024-04-24 20:12:41 +03:00)


In [1]:
# !pip install jq

In [2]:
# !pip install faiss-cpu # для gpu тут другая установка, faiss-gpu

In [4]:
CREDS = os.getenv['CREDS']

In [7]:
import json
from langchain_community.document_loaders import JSONLoader
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings.gigachat import GigaChatEmbeddings
from langchain.schema import HumanMessage, SystemMessage
from langchain.chat_models.gigachat import GigaChat

In [23]:
embeddings = GigaChatEmbeddings(
    credentials=CREDS, scope='GIGACHAT_API_CORP', verify_ssl_certs=False
)

chat = GigaChat(credentials=CREDS, scope='GIGACHAT_API_CORP', verify_ssl_certs=False)

time: 435 µs (started: 2024-04-24 20:15:18 +03:00)


In [40]:
with open('uchebnik.txt', encoding='cp1251') as file:
    data = file.read()

time: 109 ms (started: 2024-04-25 16:28:28 +03:00)


In [46]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap=0)
new_texts = text_splitter.create_documents([data])

time: 1.97 s (started: 2024-04-25 16:37:27 +03:00)


In [47]:
len(new_texts)

2310

time: 6.83 ms (started: 2024-04-25 16:37:31 +03:00)


In [48]:
new_texts[0:5]

[Document(page_content='Обеспечение обязательств (залог, поручительство, гарантия)\nРоман Сергеевич Бевзенко\n\n\nНастоящий сборник представляет собой собрание ранее опубликованных работ автора, посвященных проблематике обеспечения обязательств. В нем представлены работы по вопросам залогового права, поручительства и независимых (банковских) гарантий. Автором анализируются положения Гражданского кодекса РФ и иных законов, регулирующих обеспечение обязательств, разбирается судебная практика применения их норм.\n\nКнига представляет интерес для практикующих юристов, ученых и всех интересующихся проблемами обязательственного права и обеспечения обязательств.\n\n\n\n\n\nРоман Сергеевич Бевзенко\n\nОбеспечение обязательств (залог, поручительство, гарантия)\n\n\nПосвящается всем моим друзьям и коллегам по работе в Высшем Арбитражном Суде Российской Федерации, в общении с которыми совершенно незаметно пролетели семь лет моей жизни, которые я не забуду никогда.\n\n\n\n© Р.С. Бевзенко, 2015'),


time: 6.45 ms (started: 2024-04-25 16:37:36 +03:00)


In [49]:
new_db_faiss = FAISS.from_documents(new_texts, embeddings)

AUTHENTICATION ERROR


time: 1min 26s (started: 2024-04-25 16:38:36 +03:00)


In [50]:
query = 'Какие поправки вступили в Гражданский кодекс'
new_db_faiss.similarity_search_with_score(query, k=5)

[(Document(page_content='разной степени фундаментальности, сколько вследствие реакции судей на социальные, экономические и иные перипетии жизни общества. Если меня сегодня спросят, что такое российское гражданское право, то я отвечу так: гражданское право – это то, что суды говорят по поводу того, что написано в Гражданском кодексе РФ.'),
  255.78352),
 (Document(page_content='Кроме того, заслуживает внимания и проблема публичности такого рода «обременений» (разнообразных арестов) в контексте реформы законодательства о регистрации уведомлений о залоге движимости (об этом см. ниже).\n\n\n\n\n3.\xa0Система законодательства, регулирующего залог\n\n\nС принятием новой редакции §\xa03 гл. 23 ГК РФ система российского права, регулирующего залог, приобрела наконец определенную стройность.\n\nИтак, основная масса норм, регулирующих залог, теперь сосредоточена в названном параграфе ГК РФ. Любопытно, что по своей структуре новая редакция §\xa03 гл. 23 ГК РФ в некотором смысле представляет собой 

time: 673 ms (started: 2024-04-25 16:40:05 +03:00)


# Пример работы с Гигачат

1. В Гигачат поступают две статьи: из старой и из новой версии закона. Задача: сравнить статьи и выписать изменения.

In [69]:
old_article = '''
'Статья 349. Порядок обращения взыскания на заложенное имущество\n     \n     1. Обращение взыскания на заложенное имущество осуществляется по решению суда, если соглашением залогодателя и залогодержателя не предусмотрено обращение взыскания на заложенное имущество во внесудебном порядке.\n     2. Удовлетворение требования залогодержателя за счет заложенного имущества во внесудебном порядке допускается на основании соглашения залогодателя и залогодержателя, если иное не предусмотрено законом.\n     3. Взыскание на предмет залога может быть обращено только по решению суда в случаях, если:\n     1) предметом залога является жилое помещение, принадлежащее на праве собственности физическому лицу;\n     2) предметом залога является имущество, имеющее значительную историческую, художественную или иную культурную ценность для общества;\n     3) залогодатель - физическое лицо в установленном порядке признано безвестно отсутствующим;\n     4) заложенное имущество является предметом предшествующего и последующего залогов, при которых применяются разный порядок обращения взыскания на предмет залога или разные способы реализации заложенного имущества;\n     5) имущество заложено в обеспечение исполнения разных обязательств нескольким созалогодержателям.\n     4. Законом могут быть предусмотрены иные случаи, при которых обращение взыскания на заложенное имущество во внесудебном порядке не допускается.\n     5. В договоре о залоге, содержащем условие об обращении взыскания на заложенное имущество во внесудебном порядке или условие о порядке реализации заложенного имущества на основании решения суда, могут быть предусмотрены способ либо несколько способов реализации заложенного имущества, которые залогодержатель вправе использовать по своему усмотрению, в том числе в определенной последовательности или в зависимости от каких-либо иных условий.\n     6. Если договор о залоге, содержащий условие об обращении взыскания на заложенное имущество во внесудебном порядке, удостоверен нотариально, в случае неисполнения или ненадлежащего исполнения должником обеспеченного залогом обязательства допускается обращение взыскания на предмет залога по исполнительной надписи нотариуса без обращения в суд в порядке, установленном законодательством о нотариате и законодательством Российской Федерации об исполнительном производстве.\n     7. Особенности государственной регистрации перехода прав на заложенное недвижимое имущество, на которое обращено взыскание во внесудебном порядке, по исполнительной надписи нотариуса устанавливаются законом о государственной регистрации прав на недвижимое имущество и сделок с ним.\n     8. Если залог возникает в силу закона, обращение взыскания на заложенное имущество во внесудебном порядке возможно при условии соблюдения сторонами договора, влекущего за собой возникновение залога в силу закона, положений пунктов 5 и 6 настоящей статьи.'
'''
new_article = '''
"Статья 349. Порядок обращения взыскания на заложенное имущество
     1. Обращение взыскания на заложенное имущество осуществляется по решению суда, если соглашением залогодателя и залогодержателя не предусмотрен внесудебный порядок обращения взыскания на заложенное имущество.
     Если соглашением сторон предусмотрен внесудебный порядок обращения взыскания на заложенное имущество, залогодержатель вправе предъявить в суд требование об обращении взыскания на заложенное имущество. В этом случае дополнительные расходы, связанные с обращением взыскания на заложенное имущество в судебном порядке, возлагаются на залогодержателя, если он не докажет, что обращение взыскания на предмет залога или реализация предмета залога в соответствии с соглашением о внесудебном порядке обращения взыскания не были осуществлены в связи с действиями залогодателя или третьих лиц.
     При обращении взыскания и реализации заложенного имущества залогодержателем и иными лицами должны быть приняты меры, необходимые для получения наибольшей выручки от продажи предмета залога. Лицо, которому причинены убытки неисполнением указанной обязанности, вправе потребовать их возмещения.
     2. Удовлетворение требования залогодержателя за счет заложенного имущества без обращения в суд (во внесудебном порядке) допускается на основании соглашения залогодателя с залогодержателем, если иное не предусмотрено законом.
     3. Взыскание на предмет залога может быть обращено только по решению суда в случаях, если:
     предметом залога является единственное жилое помещение, принадлежащее на праве собственности гражданину, за исключением случаев заключения после возникновения оснований для обращения взыскания соглашения об обращении взыскания во внесудебном порядке;
     предметом залога является имущество, имеющее значительную историческую, художественную или иную культурную ценность для общества;
     залогодатель - физическое лицо в установленном порядке признано безвестно отсутствующим;
     заложенное имущество является предметом предшествующего и последующего залогов, при которых применяются разный порядок обращения взыскания на предмет залога или разные способы реализации заложенного имущества, если соглашением между предшествующим и последующим залогодержателями не предусмотрено иное;
     имущество заложено в обеспечение исполнения разных обязательств нескольким залогодержателям, за исключением случая, когда соглашением всех созалогодержателей с залогодателем предусмотрен внесудебный порядок обращения взыскания.
     Законом могут быть предусмотрены иные случаи, в которых обращение взыскания на заложенное имущество во внесудебном порядке не допускается.
     Соглашения, заключенные с нарушением требований настоящего пункта, ничтожны.
     4. Стороны вправе включить условие о внесудебном порядке обращения взыскания в договор залога.
     5. Соглашение об обращении взыскания во внесудебном порядке на заложенное имущество должно быть заключено в той же форме, что и договор залога этого имущества.
     6. Обращение взыскания на предмет залога по исполнительной надписи нотариуса без обращения в суд допускается в порядке, установленном законодательством о нотариате и законодательством Российской Федерации об исполнительном производстве, в случае неисполнения или ненадлежащего исполнения должником обеспеченного залогом обязательства, если договор залога, содержащий условие об обращении взыскания на заложенное имущество во внесудебном порядке, удостоверен нотариально.
     7. Соглашение об обращении взыскания на заложенное имущество во внесудебном порядке должно содержать указание на один способ или несколько способов реализации заложенного имущества, предусмотренных настоящим Кодексом, а также стоимость (начальную продажную цену) заложенного имущества или порядок ее определения.
     В случае, если соглашение об обращении взыскания на заложенное имущество предусматривает несколько способов реализации заложенного имущества, право выбора способа реализации принадлежит залогодержателю при условии, что соглашением не предусмотрено иное.
     8. Если обращение взыскания на заложенное имущество осуществляется во внесудебном порядке, залогодержатель или нотариус, который производит обращение взыскания на заложенное имущество в порядке, установленном законодательством о нотариате, обязан направить залогодателю, известным им залогодержателям, а также должнику уведомление о начале обращения взыскания на предмет залога.
     Реализация заложенного имущества допускается не ранее чем через десять дней с момента получения залогодателем и должником уведомления залогодержателя или нотариуса, если иной срок не предусмотрен законом, а также если больший срок не предусмотрен соглашением между залогодержателем и залогодателем. В случаях, предусмотренных банковским законодательством, реализация заложенного движимого имущества может быть осуществлена до истечения указанного срока при существенном риске значительного снижения стоимости предмета залога по сравнению с ценой реализации (начальной продажной ценой), указанной в уведомлении."
'''

messages = [
    SystemMessage(
        content="Ты - квалифицированный юрист, который умеет анализировать статьи законов и внесенные в них изменения."
    )
]
messages.append(HumanMessage(content=f'''
Привет! Сравни две итерации фрагмента закона и опиши юридическим языком по пунктам, в чем произошли изменения:
Старая версия закона: {old_article}
Новая версия закона: {new_article}
Суммаризируй ответ, опиши только значимые изменения, кратко.
'''
                            ))

res = chat(messages)
print(res.content)

AUTHENTICATION ERROR


В новой версии закона изменены положения о порядке обращения взыскания на заложенное имущество. Теперь обращение взыскания на заложенное имущество осуществляется по решению суда, если соглашением залогодателя и залогодержателя не предусмотрен внесудебный порядок обращения взыскания на заложенное имущество. Также введена норма о дополнительных расходах, связанных с обращением взыскания на заложенное имущество в судебном порядке, которые возлагаются на залогодержателя. Кроме того, в новой версии закона уточнено, что соглашения, заключенные с нарушением требований, являются ничтожными.
time: 2.6 s (started: 2024-04-25 23:35:10 +03:00)


In [70]:
messages = [
    SystemMessage(
        content="Ты - квалифицированный юрист, который умеет анализировать статьи законов и внесенные в них изменения."
    )
]
messages.append(HumanMessage(content=f'''
Привет! Сравни две редакции одной и той же статьи закона, принятые в разное время, и прокомментируй, используя юридическую терминалогию, в чем состояли изменения.
Старая версия закона: {old_article}
Новая версия закона: {new_article}
Суммаризируй ответ, опиши только значимые изменения, кратко.
'''
                            ))

res = chat(messages)
print(res.content)

В новой версии статьи 349 закона были внесены следующие изменения:

1. Уточнен порядок обращения взыскания на заложенное имущество. Теперь оно осуществляется по решению суда, если стороны не предусмотрели внесудебный порядок.

2. Установлено, что дополнительные расходы, связанные с обращением взыскания на заложенное имущество в судебном порядке, возлагаются на залогодержателя, если он не докажет, что обращение взыскания на предмет залога или реализация предмета залога в соответствии с соглашением о внесудебном порядке обращения взыскания не были осуществлены в связи с действиями залогодателя или третьих лиц.

3. Уточнены случаи, при которых обращение взыскания на заложенное имущество во внесудебном порядке не допускается.

4. Установлено, что соглашения, заключенные с нарушением требований настоящего пункта, ничтожны.

5. Уточнены условия и порядок заключения соглашения об обращении взыскания во внесудебном порядке на заложенное имущество.

6. Уточнены основания и порядок обращения взы

2. В векторную базу направляется запрос по каждому из пунктов и вытаскиваются наиболее близкие фрагменты из учебника, для подготовки экспертного комментария.

In [71]:
query = '''В новой версии статьи 349 закона были внесены следующие изменения:

1. Уточнен порядок обращения взыскания на заложенное имущество. Теперь оно осуществляется по решению суда, если стороны не предусмотрели внесудебный порядок.

2. Установлено, что дополнительные расходы, связанные с обращением взыскания на заложенное имущество в судебном порядке, возлагаются на залогодержателя, если он не докажет, что обращение взыскания на предмет залога или реализация предмета залога в соответствии с соглашением о внесудебном порядке обращения взыскания не были осуществлены в связи с действиями залогодателя или третьих лиц.

3. Уточнены случаи, при которых обращение взыскания на заложенное имущество во внесудебном порядке не допускается.

4. Установлено, что соглашения, заключенные с нарушением требований настоящего пункта, ничтожны.

5. Уточнены условия и порядок заключения соглашения об обращении взыскания во внесудебном порядке на заложенное имущество.

6. Уточнены основания и порядок обращения взыскания на заложенное имущество по исполнительной надписи нотариуса.

7. Уточнены сроки и порядок реализации заложенного имущества.'''
expertise = new_db_faiss.similarity_search_with_score(query, k=5)
expertise

AUTHENTICATION ERROR


[(Document(page_content='Даже если соглашением сторон предусмотрен внесудебный порядок обращения взыскания на заложенное имущество, залогодержатель тем не менее вправе воспользоваться судебным порядком обращения взыскания, т.\xa0е. предъявить в суд требование об обращении взыскания на заложенное имущество. Однако в этом случае дополнительные расходы, связанные с обращением взыскания на заложенное имущество в судебном порядке, вне зависимости от исхода процесса возлагаются на залогодержателя. Исключение составляет случай, когда залогодержатель докажет, что обращение взыскания на предмет залога или реализация предмета залога в соответствии с соглашением о внесудебном порядке обращения взыскания не были осуществлены в связи с действиями залогодателя или третьих лиц (абзац второй п. 1\xa0ст. 349 ГК РФ).'),
  112.05083),
 (Document(page_content='Соглашение об обращении взыскания на заложенное имущество во внесудебном порядке должно содержать указание на один или несколько способов реализаци

time: 920 ms (started: 2024-04-25 23:41:12 +03:00)


3. В Гигачат направляются результаты выдачи FAISS и тезисы об изменениях. Задача: дать экспертно-правовой комментарий, основываясь на этих пунктах из учебника и тезисах.

In [72]:
messages = [
    SystemMessage(
        content="Ты - квалифицированный юрист, который умеет анализировать статьи законов и внесенные в них изменения."
    )
]
messages.append(HumanMessage(content=f'''
Напиши экспертно-правовой комментарий об изменениях в статье закона, основываясь на выдержках из научной работы "Обеспечение обязательств (залог, поручительство, гарантия)" Романа Сергеевича Бевзенко.
Изменения в законе: {query}
Выдержки из научной работы: {expertise}
В ответе делай отсылку к научной работе. Ответ юридическим языком:
'''
                            ))

res = chat(messages)
print(res.content)

Изменения в статье 349 закона, касающиеся порядка обращения взыскания на заложенное имущество, были внесены с целью уточнения процедуры и распределения расходов. Теперь обращение взыскания на заложенное имущество осуществляется по решению суда, если стороны не предусмотрели внесудебный порядок. Дополнительные расходы, связанные с обращением взыскания на заложенное имущество в судебном порядке, возлагаются на залогодержателя, если он не докажет, что обращение взыскания на предмет залога или реализация предмета залога в соответствии с соглашением о внесудебном порядке обращения взыскания не были осуществлены в связи с действиями залогодателя или третьих лиц. Уточнены случаи, при которых обращение взыскания на заложенное имущество во внесудебном порядке не допускается. Соглашения, заключенные с нарушением требований настоящего пункта, ничтожны. Уточнены условия и порядок заключения соглашения об обращении взыскания во внесудебном порядке на заложенное имущество. Уточнены основания и поряд

# Сохраняем FAISS и демонстрируем работу подгруженного FAISS

In [61]:
# сохраняем FAISS

FOLDER_PATH_INDEX = "faiss_trained_on_dataset"
NAME_INDEX = "langchains_index"

new_db_faiss.save_local(folder_path=FOLDER_PATH_INDEX, index_name=NAME_INDEX)

time: 283 ms (started: 2024-04-25 19:46:07 +03:00)


In [ ]:
db = FAISS.from_documents(texts, model_fine_tune)

In [64]:
# загружаем FAISS и настраиваем работу

from langchain_community.vectorstores import FAISS
from langchain_community.embeddings.gigachat import GigaChatEmbeddings

FOLDER_PATH_INDEX = "faiss_trained_on_dataset"
NAME_INDEX = "langchains_index"

# количество соседей для кажого ретривера
NEIGHBOUR = 5

embeddings = GigaChatEmbeddings(
    credentials=CREDS, scope='GIGACHAT_API_CORP', verify_ssl_certs=False
)

faiss_db = FAISS.load_local(folder_path=FOLDER_PATH_INDEX, embeddings=embeddings, index_name=NAME_INDEX, allow_dangerous_deserialization=True)
faiss_db

time: 159 ms (started: 2024-04-25 19:53:15 +03:00)


In [65]:
# проверка работы

query = 'В старой версии закона говорится о возможности обращения взыскания на заложенное имущество по решению суда или во внесудебном порядке. В новой версии уточняется, что обращение взыскания на заложенное имущество осуществляется по решению суда, если стороны не предусмотрели внесудебный порядок.'
expertise = new_db_faiss.similarity_search_with_score(query, k=5)
expertise

AUTHENTICATION ERROR


[(Document(page_content='Соглашение об обращении взыскания на заложенное имущество во внесудебном порядке должно содержать указание на один или несколько способов реализации заложенного имущества, предусмотренных ГК РФ, а также стоимость (начальную продажную цену) заложенного имущества или порядок ее определения.\n\nВ случае, если соглашение об обращении взыскания на заложенное имущество предусматривает несколько способов реализации заложенного имущества, право выбора способа реализации принадлежит залогодержателю при условии, что соглашением не предусмотрено иное (п. 7\xa0ст. 349 ГК РФ).\n\nЗаконодатель изменил перечень случаев, когда внесудебное обращение взыскания не допускается (п. 3\xa0ст. 349 ГК РФ):'),
  159.77779),
 (Document(page_content='34.\xa0Действующая редакция ст. 349 ГК РФ предусматривает, что по общему правилу взыскание на предмет залога (вне зависимости от вида заложенного имущества) обращается судом. Однако стороны договора о залоге вправе заключить соглашение о поря

time: 1.21 s (started: 2024-04-25 19:53:40 +03:00)
